### chaptGPT specs   

A decoder-only transformer in pytorch to predict 'next output' at each time step. 

Each time step t is represented by a vector of n=4 tokens. 
The length of the sequence (context window) is Ti=86 for inference, and Tt=8*Ti for training. That is, the context window for training is 8 times the length of the context window for inference. 
The attention is "causal", looking only back in time, and the maximum look-back time for the attention blocks is Ti (even when the sequence is longer during training).

The size of the vocabulary for each of the n tokens is V=1024. 

The dataloader will as is usual, supply batches pairs of (input,target) where the size of each input and output is Tt*n (the sequence length times the number of tokens at each time step). The tokens are indexes for the vocabulary in the range of (0, V-1). The targets are shifted relative to the input sequence by 1 as is typical for networks the learn to predict the output for the next time step. 

The first layer in the architecture will be a learnable "multiembedding" layer that embeds each of the 4 tokens at each time step as an 8-dimensional vector. The n 8-dimensional vectors is concatenated to provide the 32 dimensional input for the transformer blocks at each time step. 

A positional code is then added to each 32 dimensional vector. For positional encoding, use Rotary Position Embedding (RoPE).

We use a stack of b=8 transformer blocks that are standard (using layer norms, a relu for activation, and a forward expansion factor of 4 form the linear layer). Each transformer block consumes and produces a context window length sequence of 32 dimensional vectors. 

After the last transformer block, there should be linear layer that maps the 32 dimensional vectors to the output size which is V*n (the vocabulary size time the number of tokens stacked at each time step). These are the logits that will be fed to the softmax functions (one for each of the n tokens) that provide the probability distribtion across the vocabulary set. Use the criterion nn.CrossEntropyLoss() for computing the loss using the targets provided by the dataloader, and Adam for the optimizer.

Again, at inference time, the fixed-length context window is shorter than the training sequence window length, and equal to the maximum look-back time of the attention blocks. The inference process should take the output produced at each time step (a stack of n tokens), and shift them in to a sliding window that is used for input for the next time step. Please use the "Incremental Token Generation Using Cached States" approach to minimize the computational burden of the shifting input window. Please be careful that it works correctly with the RoPE positional coding. The length of the sequences generated during inference is arbitrary and should be settable with a parameter. 

Include an example (with dummy inputs and targets) of how to call the code for a training step, and an example of inference for a specifiable output length.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time

import numpy as np

# and for creating a custom dataset and loader:
from torch.utils.data import DataLoader
import os


from utils.utils import generate_mask, save_model, writeDACFile, interpolate_vectors
from DACTransformer.DACTransformer import TransformerDecoder
from dataloader.dataset import CustomDACDataset, onehot, getNumClasses

In [2]:
from torch.utils.tensorboard import SummaryWriter

### <font color='blue'> Parameters 
</font>

In [3]:
# Training data dir

testsnd="bees" # pistons, wind, applause, bees
# data_dir="/scratch/syntex/PisWinAppBee_long_44/onesnddac/"+testsnd+"-train-small"  ##******* small
# validator_data_dir="/scratch/syntex/PisWinAppBee_long_44/onesnddac/"+testsnd+"-val-small"

data_dir=validator_data_dir="/scratch/syntex/PisWinAppBee_long_44/onesnddac/"+testsnd+"-train-small"
validator_data_dir="/scratch/syntex/PisWinAppBee_long_44/onesnddac/"+testsnd+"-val-small"

# ---------     for the transformr  --------------#
vocab_size = 1024
num_tokens = 4

cond_classes = 7 #getNumClasses()
cond_params = 1 #1
cond_size = cond_classes + cond_params # num_classes + num params - not a FREE parameter!
embed_size = 512 -cond_size # 240 #32  # embed_size must be divisible by num_heads and by num tokens
Ti = 86
Tt = 430 # must match the length of the sequences in the batch
batch_size = 6
sequence_length = Tt  # For training

num_layers=2
num_heads=4 # 8 # embed_size must be divisible by num_heads
forward_expansion=4 #4
dropout_rate=0.1
learning_rate=0.001

top_n = 5   # not used yet

num_epochs=100
experiment_name='newscratch' + testsnd + 'cond' + str(cond_size)  #the higher the embed size (to 240 anyway) the fewer epochs are necessary
outdir = 'runs' + '/' + experiment_name
basefname=testsnd+ '.e' + str(embed_size) + '.l' + str(num_layers) + '.h' + str(num_heads) 

DEVICE='cuda'#My experiments show CUDA is only) 4 times faster than CPU!
ALSO_RUN_DUMMY_STEP =False # there a two cells that setup a single dummy input and target and take one training step

inference_steps=86*10 # second fact is number of seconds (for 86 tokens/sec)

ErrorLogRate=10
checkpoint_interval=25

# For inference (does an interpolation between min to max and back to min)
infminpval=.5
infmaxpval=.6

print(f'basefname = {basefname}')
print(f'outdir = {outdir}')

basefname = bees.e504.l2.h4
outdir = runs/newscratchbeescond8


### <font color='blue'> Set up cuda. 
Without it, training runs about 4 times slower  
</font>

In [4]:
torch.cuda.device_count()
torch.cuda.get_device_properties(0).total_memory/1e9

device = torch.device(DEVICE) # if the docker was started with --gpus all, then can choose here with cuda:0 (or cpu)
torch.cuda.device_count()
print(f'memeory on cuda 0 is  {torch.cuda.get_device_properties(0).total_memory/1e9}')

device

memeory on cuda 0 is  25.216745472


device(type='cuda')

### <font color='blue'> Load data 
</font>

In [5]:
# Create an instance of the dataset
dataset = CustomDACDataset(data_dir=data_dir)
# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#Validator data set
if validator_data_dir != None :
    validator_dataset=CustomDACDataset(data_dir=validator_data_dir)
    validator_dataloader= DataLoader(validator_dataset, batch_size=batch_size, shuffle=True)

#---------------------------------------------------------------
# Test data dir
for batch_idx, (inputs, targets, cvect) in enumerate(dataloader):
    #pass
    # Your training code here
    # inputs: batch of input data of shape [batch_size, N, T-1]
    # targets: corresponding batch of target data of shape [batch_size, N, T-1]
    
    if (batch_idx == 0) : 
        print(f"Batch {batch_idx + 1}")
        print(f"Inputs shape: {inputs.shape}")
        print(f"Targets shape: {targets.shape}")

Batch 1
Inputs shape: torch.Size([6, 430, 4])
Targets shape: torch.Size([6, 430, 4])


### <font color='blue'> Instantiate model 
</font>

In [6]:
mask = generate_mask(Tt, Ti).to(device)
print(f'Mask.shape is {mask.shape}')
mask

Mask.shape is torch.Size([430, 430])


tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [-inf, -inf, -inf,  ..., 0., -inf, -inf],
        [-inf, -inf, -inf,  ..., 0., 0., -inf],
        [-inf, -inf, -inf,  ..., 0., 0., 0.]], device='cuda:0')

In [7]:
# Instantiate model, put it on the device
#model = TransformerDecoder(embed_size, num_layers, num_heads, forward_expansion, dropout_rate, Tt, num_tokens, vocab_size).to(device)
print(f'Creating model with embed_size={embed_size}, cond_size={cond_size}')
model = TransformerDecoder(embed_size, num_layers, num_heads, forward_expansion, dropout_rate, Tt, num_tokens, vocab_size, cond_size).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

Creating model with embed_size=504, cond_size=8
Get a coder with embed_size=504. cond_size=8, max_len=430


In [8]:
def inference(model, inference_cond, Ti, vocab_size, num_tokens, inference_steps, fname) :
    model.eval()
    mask = generate_mask(Ti, Ti).to(device)
    input_data = torch.randint(0, vocab_size, (1, Ti, num_tokens)).to(device)  # Smaller context window for inference
    predictions = []

    t0 = time.time()
    for i in range(inference_steps):  # Generate 100 tokens
        if cond_size == 0:
            output = model(input_data, None, mask) # step through 
        else : 
            output = model(input_data, inference_cond[:, i:Ti+i, :], mask) # step through

        # This takes the last vector of the sequence (the new predicted token stack) so has size(b,1,4,1024)
        # This it takes the max across the last dimension (scores for each element of the vocabulary (for each of the 4 tokens))
        # .max returns a duple of tensors, the first are the max vals (one for each token) and the second are the
        #        indices in the range of the vocabulary size. 
        # THAT IS, the 4 selected "best" tokens are taken independently
        next_token = output[:, -1, :, :].max(-1)[1]  # Greedy decoding for simplicity
        predictions.append(next_token)
        input_data = torch.cat([input_data, next_token.unsqueeze(1)], dim=1)[:, 1:]  # Slide window

    t1 = time.time()
    inf_time = t1-t0
    print(f'inference time for {inference_steps} steps, or {inference_steps/86} seconds of sound is {inf_time}' )

    dacseq = torch.cat(predictions, dim=0).unsqueeze(0).transpose(1, 2)
    if mask == None:
        writeDACFile(fname + '_unmasked', dacseq)
    else :
        writeDACFile(fname, dacseq)       

In [9]:
if cond_size == 0 :
    inference_cond = None
else : 
    one_hot_fvector=onehot(testsnd)
    print(f'one_hot_fvector({testsnd}) = {one_hot_fvector}')

    cvect1=torch.cat((one_hot_fvector, torch.tensor([infminpval])))
    cvect2=torch.cat((one_hot_fvector, torch.tensor([infmaxpval])))

    steps=[0,Ti+inference_steps//2, Ti+inference_steps]
    inference_cond=interpolate_vectors([cvect1,cvect2, cvect1 ], steps) #length must cover staring context window+inf steps

    print(f'cvect1 = {cvect1}')
    print(f'cvect2 = {cvect2}')

    inference_cond=inference_cond.to(device)

    print(f'shape of inf_cond is  = {inference_cond.shape}') 

one_hot_fvector(bees) = tensor([0., 0., 0., 1., 0., 0., 0.])
cvect1 = tensor([0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.5000])
cvect2 = tensor([0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.6000])
shape of inf_cond is  = torch.Size([1, 947, 8])


/transformerlw/utils/utils.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_tensors = [torch.tensor(vec) for vec in v]


### <font color='blue'> Train !! 
</font>

In [10]:
# Initialize SummaryWriter
writer = SummaryWriter(outdir)


In [11]:

t0 = time.time()

for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    model.train()
    for batch_idx, (input_data, target_data, cond_data) in enumerate(dataloader):
        #print(f"b{batch_idx} ", end='')
        optimizer.zero_grad()

        # Move inputs and targets to the device
        input_data, target_data, cond_data = input_data.to(device), target_data.to(device), cond_data.to(device)
        
        if cond_size==0 :  #Ignore conditioning data
            cond_expanded=None
        else : 
            # for dataset exammples, expand the conditioning info across all time steps before passing to models
            cond_expanded = cond_data.unsqueeze(1).expand(-1, input_data.size(1), -1)
        
        #print(f'    after loading a batch,  input_data.shape is {input_data.shape}, and cond_data.shape is {cond_data.shape}')
        #print(f'    after loading a batch,  cond_expanded.shape is {cond_expanded.shape}')

        output = model(input_data, cond_expanded, mask)
        loss = criterion(output.reshape(-1, vocab_size), target_data.reshape(-1)) # collapses all target_data dimensions into a single dimension

        loss.backward()
        optimizer.step()
    if (epoch+1) % ErrorLogRate == 0:
        print(f'')
        print(f'EPOCH {epoch+1}  ', end='')
        print(f'loss: {loss}')
        # Log the loss to TensorBoard
        writer.add_scalar('Loss/train', loss, epoch)
        
        if validator_data_dir != None :
            model.eval()
            with torch.no_grad():
                val_loss = 0
                for val_inputs, val_targets, cond_data in validator_dataloader:
                    val_inputs, val_targets, cond_data = val_inputs.to(device), val_targets.to(device), cond_data.to(device)
                    
                    if cond_size==0 :  #Ignore conditioning data
                        cond_expanded=None
                    else: 
                        # for dataset exammples, expand the conditioning info across all time steps before passing to models
                        cond_expanded = cond_data.unsqueeze(1).expand(-1, input_data.size(1), -1)

                    
                    val_outputs = model(val_inputs,cond_expanded, mask)
                    
                    val_loss += criterion(val_outputs.reshape(-1, vocab_size), val_targets.reshape(-1)) # collapses all target_data dimensions into a single dimension
                    #val_loss += criterion(val_outputs, val_targets).item()

            print(f'Validation Loss: {val_loss / len(validator_dataloader)}')
            writer.add_scalar('Loss/validation', val_loss, epoch)
            
    if (epoch+1) % checkpoint_interval == 0:
        lastbasename = outdir+"/"+basefname+"_chkpt_"+str(epoch+1).zfill(4)
        save_model(model, Ti,  lastbasename +".pth")
        inference(model, inference_cond, Ti, vocab_size, num_tokens, inference_steps, lastbasename) 
            
t1 = time.time()
train_time = t1-t0
print(f'train time for {num_epochs} epochs, was {train_time}' )

    


EPOCH 10  loss: 3.401169776916504
Validation Loss: 5.4217939376831055

EPOCH 20  loss: 1.9584871530532837
Validation Loss: 7.181422233581543
inference time for 860 steps, or 10.0 seconds of sound is 1.325376272201538

EPOCH 30  loss: 1.2794727087020874
Validation Loss: 8.759988784790039

EPOCH 40  loss: 0.9572731852531433
Validation Loss: 9.908573150634766

EPOCH 50  loss: 0.7961196303367615
Validation Loss: 10.644936561584473
inference time for 860 steps, or 10.0 seconds of sound is 1.3506247997283936

EPOCH 60  loss: 0.7812052965164185
Validation Loss: 11.274145126342773

EPOCH 70  loss: 0.6596552133560181
Validation Loss: 11.545555114746094
inference time for 860 steps, or 10.0 seconds of sound is 1.3205788135528564

EPOCH 80  loss: 0.6191518306732178
Validation Loss: 11.925834655761719

EPOCH 90  loss: 0.5793048143386841
Validation Loss: 11.979930877685547

EPOCH 100  loss: 0.5267612338066101
Validation Loss: 12.145916938781738
inference time for 860 steps, or 10.0 seconds of soun

In [12]:
#just check that inference attention mask looks right
#Actually, the inference mask can be None since we are using a context window only as long as the maximum look-back in the training mask
# thats why taking the mask with :TI is upper-triangular. Longer dims would show a banded mask again.
foo=mask[:Ti, :Ti]
foo

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

### <font color='blue'> User DAC2Audio.ipynb  
to see and hear your generated audio   
</font>

In [13]:
foo=interpolate_vectors([[1,2,3], [0,0,0], [1,1,1], [3,2,1]], [0,3,4,7])
foo
foo.shape

torch.Size([1, 8, 3])

In [14]:
v = [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
s = [0, 2, 4]
result_tensor = interpolate_vectors(v, s)
print(result_tensor)

tensor([[[1., 2.],
         [2., 3.],
         [3., 4.],
         [4., 5.],
         [5., 6.]]])
